In [17]:
import os
import json
import pandas as pd
import traceback

In [18]:
from langchain.chat_models import ChatOpenAI

In [19]:
key=os.getenv("OPENAI_API_KEY")

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [22]:
import PyPDF2

In [25]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [27]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [30]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [42]:
TEMPLATE ="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [35]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [37]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [38]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [39]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [40]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [41]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [44]:
file_path=r"C:\Users\gauta\mcqgenAI\experiment\data.txt"

In [46]:
file_path

'C:\\Users\\gauta\\mcqgenAI\\experiment\\data.txt'

In [45]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [47]:
print(TEXT)

Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin; lit. 'The Advancing Giant') is a Japanese manga series written and illustrated by Hajime Isayama. It is set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans; the story follows Eren Yeager, who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha's monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.

An anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episode third season was broadcast in two parts, with the first 12 episodes airing from July to October 2018 and 

In [48]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [49]:
NUMBER=5
SUBJECT='ATTACK ON TITAN'
TONE='SIMPLE'

In [50]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\gauta\AppData\Local\Temp\ipykernel_16336\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin; lit. 'The Advancing Giant') is a Japanese manga series written and illustrated by Hajime Isayama. It is set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans; the story follows Eren Yeager, who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha's monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.

An anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episo

In [51]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1995
Prompt Tokens:1298
Completion Tokens:697
Total Cost:0.0033410000000000002


In [52]:
response

{'text': "Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin; lit.\u2009'The Advancing Giant') is a Japanese manga series written and illustrated by Hajime Isayama. It is set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans; the story follows Eren Yeager, who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha's monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.\n\nAn anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episode third season was broadcast in two parts, with the first 12 episodes airing from July to 

In [54]:
quiz=response.get("quiz")

In [56]:
quiz=json.loads(quiz)

In [57]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [69]:
quiz_table_data

[{'MCQ': 'Who is the main character in Attack on Titan?',
  'Choices': 'a: Mikasa Ackerman | b: Levi Ackerman | c: Eren Yeager | d: Armin Arlert',
  'Correct': 'c'},
 {'MCQ': 'What is the name of the city where humanity lives surrounded by walls?',
  'Choices': 'a: Trost | b: Shiganshina | c: Maria | d: Wall Rose',
  'Correct': 'b'},
 {'MCQ': 'Who does Eren Yeager vow to exterminate in Attack on Titan?',
  'Choices': 'a: Titans | b: Humans | c: Animals | d: Plants',
  'Correct': 'a'},
 {'MCQ': 'Which studio produced the first three seasons of the Attack on Titan anime?',
  'Choices': 'a: Bones | b: Wit Studio | c: Madhouse | d: Studio Ghibli',
  'Correct': 'b'},
 {'MCQ': 'How many tankōbon volumes have collected the chapters of Attack on Titan manga by April 2021?',
  'Choices': 'a: 20 | b: 30 | c: 34 | d: 40',
  'Correct': 'c'}]

In [62]:
quiz=pd.DataFrame(quiz_table_data)

In [65]:
quiz.to_csv("Attack_on_titan.csv",index=False)

In [67]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_03_2025_15_59_38'

In [70]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_03_2025_19_47_00'

In [1]:
from langchain_community.callbacks.manager import get_openai_callback

print("Import worked!")

Import worked!
